# 103590450 馬茂源 四資四

In [44]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import download
import time
import os
import sys
import string
import html
import hashlib
import re
from collections import defaultdict

In [26]:
#download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brian41005/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
t0 = time.time()

In [10]:
if not os.path.exists('result'):
    os.mkdir('result')
for i in range(1, 5):
    dir_ = 'result/task{}'.format(i)
    if not os.path.exists(dir_):
        os.mkdir(dir_)

In [11]:
file_names = ['./data/reut2-{0:0>3}.sgm'.format(i) for i in range(22)]
file_names = [file_names[0]]

## (1) Given the Reuters-21578 dataset, please calculate all kshingles and output the set representation of the text dataset as a matrix. 

In [12]:
def parser(file_name):
    with open(file_name, 'r', encoding='ISO-8859-1') as f:
        file = f.read()
    news = []
    start = 0
    for i in range(len(file)):
        if file[i:i+6] == '<BODY>':
            start = i+6
        elif file[i:i+7] == '</BODY>':
            n = file[start:i].replace('\n', ' ')
            n = n.replace('REUTER &#3;', '')
            news.append(n)
    return news

In [13]:
news = []
for i in file_names:
    each_news = parser(i) 
    news.extend(each_news)

In [14]:
len(news)

925

In [15]:
news_data = pd.DataFrame(data=news, columns=['news'])

In [16]:
news_data.head()

,news
0,Showers continued throughout the week in the B...
1,Standard Oil Co and BP North America Inc said ...
2,Texas Commerce Bancshares Inc's Texas Commerce...
3,BankAmerica Corp is not under pressure to act ...
4,The U.S. Agriculture Department reported the f...


In [49]:
def tokenizer(text):
    strip_chars = '.' + ' –…' + string.punctuation
    text = html.unescape(text)
    text = text.lower()
    text = text.strip(strip_chars)
    text = re.sub(re.compile('<.*?>'), '', text)
#     result = []
#     for token in html.unescape(str(text)).split():
#         token = token.lower()
#         token = token.strip(strip_chars)
#         if not token.isalpha():
#             continue
#         length = len(token)
#         if length < 1 or length > 26:
#             continue
#         result.append(token)
    return text

In [42]:
test_news = news_data['news'][1]

In [50]:
tokenizer(test_news)


'standard oil co and bp north america inc said they plan to form a venture to manage the money market borrowing and investment activities of both companies.     bp north america is a subsidiary of british petroleum co plc , which also owns a 55 pct interest in standard oil.     the venture will be called bp/standard financial trading and will be operated by standard oil under the oversight of a joint management committee.   reuter'

In [33]:
sent_tokenize(news_data['news'][1])

['Standard Oil Co and BP North America Inc said they plan to form a venture to manage the money market borrowing and investment activities of both companies.',
 'BP North America is a subsidiary of British Petroleum Co Plc &lt;BP>, which also owns a 55 pct interest in Standard Oil.',
 'The venture will be called BP/Standard Financial Trading and will be operated by Standard Oil under the oversight of a joint management committee.',
 'Reuter &#3;']

In [20]:
news_data['news_token'] = news_data.apply(lambda x: sent_tokenize(x['news']), axis=1)

LookupError: ("\n**********************************************************************\n  Resource \x1b[93mpunkt\x1b[0m not found.\n  Please use the NLTK Downloader to obtain the resource:\n\n  \x1b[31m>>> import nltk\n  >>> nltk.download('punkt')\n  \x1b[0m\n  Searched in:\n    - '/Users/brian41005/nltk_data'\n    - '/usr/share/nltk_data'\n    - '/usr/local/share/nltk_data'\n    - '/usr/lib/nltk_data'\n    - '/usr/local/lib/nltk_data'\n    - '/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/nltk_data'\n    - '/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/nltk_data'\n    - ''\n**********************************************************************\n", 'occurred at index 0')

In [ ]:
news_data['news_token']

In [12]:
def k_shingle(text, k):
    string = ' '.join(tokenizer(text))
    shingles = set([])
    for i in range(len(string)-k + 1):
        shingles.add(string[i:i+k])
    return shingles

In [13]:
news_data['news_token'] = news_data.apply(lambda x: k_shingle(x['news_token'], 5), axis=1)

In [14]:
shingles = set([])
for s in news_data['news_token'].values:
    shingles |= s

In [15]:
N = len(shingles)
N

60332

In [16]:
def encode_shingles(row, shingles):
    vector = np.zeros(len(shingles), dtype='int')
    for shingle, idx in zip(shingles, range(len(shingles))):
        if shingle in row:
            vector[idx] = 1
    return vector

In [17]:
news_data['shingles'] = news_data['news_token'].apply(encode_shingles, args=(shingles,))

In [18]:
output_shingles = np.array(news_data['shingles'].tolist()).T.astype('int')

In [19]:
#np.savetxt("task_1.csv", output_shingles, delimiter=",",  fmt='%d')

## (2) Given the set representation, compute the minhash signatures of all documents using MapReduce.

In [20]:
news_shingles = output_shingles

In [21]:
def get_prime(greater_than):
    
    def is_prime(n):
        if n % 2 == 0 and n > 2: 
            return False
        return all(n % i for i in range(3, int(np.sqrt(n)) + 1, 2))
    is_p = False
    
    
    
    while not is_p:
        greater_than += 1
        is_p = is_prime(greater_than)
        
    return greater_than

In [22]:
def get_hash_func_list(n, k=100):
    p = get_prime(n)
    func_list = []
    for a, b in zip(np.random.randint(0, n, size=k),
                   np.random.randint(0, n, size=k)):
        func_list.append(lambda x, a=a,b=b,p=p,n=n: ((a*x+b)%p)%n)
    return np.array(func_list)

In [23]:
def one_pass_minhashing(shingles, k=100):
    n = shingles.shape[0]
    hash_list = get_hash_func_list(n, k=k)
    singnature = np.full((k, shingles.shape[1]), fill_value=np.inf)
    
    for i in range(n):
        hash_value = np.array([h(i) for h in hash_list])

        for j, c in enumerate(shingles[i, :] == 1):
            if c:
                mask = singnature[:, j] > hash_value
                singnature[:, j][mask] = hash_value[mask]
            
    return singnature

In [24]:
test_input = np.array([[1,0,1,0],
                       [1,0,0,1],
                       [0,1,0,1],
                       [0,1,0,1],
                       [0,1,0,1],
                       [1,0,1,0],
                       [1,0,1,0]])
test_singnature = one_pass_minhashing(test_input, k=6)
test_singnature.astype('int')

array([[0, 0, 1, 0],
       [0, 3, 0, 2],
       [0, 1, 0, 0],
       [1, 2, 1, 2],
       [0, 1, 1, 0],
       [1, 0, 2, 0]])

In [25]:
 #np.unique((minhashing[:, 0] - minhashing[:, 2]) == 0, return_counts=True)

In [26]:
singnature = one_pass_minhashing(news_shingles, k=100)

In [27]:
singnature.shape

(100, 925)

In [28]:
#np.savetxt("task_2.csv", singnature, delimiter=",",  fmt='%d')

## (3) Implement the LSH algorithm by MapReduce and output the resulting candidate pairs of similar documents.

In [29]:
hashlib.sha512().block_size

128

In [30]:
def LSH(singnature, b=2):
    bucket = defaultdict(list)
    k = singnature.shape[0]
    for i, doc in enumerate(singnature.T):
        x = np.array2string(doc.astype('int'), separator='', precision=0)
        for j in range(0, k, b):
            #print(j, j+b)
            key = hashlib.sha512(x[j:j+b].encode()).hexdigest()
            bucket[key].append(i)
    return bucket

In [31]:
def remove_alone(bucket):
    remove_list = []
    for k, item in bucket.items():
        if len(item) == 1:
            remove_list.append(k)
    for k in remove_list:
        bucket.pop(k)
    return bucket

In [32]:
# bucket = LSH(test_singnature, b=2)
# bucket

In [33]:
bucket = LSH(singnature, b=5)
bucket = remove_alone(bucket)
print(len(bucket))

3058


In [37]:
for k, item in bucket.items():
#     if len(item) == 2:
    print(item)
        #pass

[0, 56, 96, 108, 128, 182, 216, 240, 247, 254, 324, 371, 452, 460, 712, 776]
[0, 182, 323]
[0, 17, 29, 52, 308, 758]
[0, 218, 239, 390, 582, 609, 611, 662, 779]
[0, 589]
[0, 27, 185, 345]
[0, 29, 52, 513, 691, 823, 888]
[0, 54, 162, 253, 262, 297, 340, 342, 359, 711, 780, 897]
[0, 152]
[0, 280]
[0, 182, 743, 858]
[0, 524]
[0, 187, 408, 664]
[0, 94, 179, 403, 793, 863, 877]
[0, 241, 890]
[0, 3, 15, 271, 280, 294, 320, 335, 342, 791, 843, 895, 899]
[1, 8, 117, 130, 257, 444, 555, 604]
[1, 8, 16, 30, 31, 32, 49, 61, 64, 71, 81, 99, 103, 118, 132, 138, 142, 148, 149, 151, 152, 154, 168, 169, 171, 266, 364, 379, 380, 413, 482, 489, 521, 537, 545, 546, 555, 599, 600, 601, 604, 610, 617, 627, 634, 649, 657, 673, 689, 694, 707, 718, 720, 727, 731, 749, 784, 787, 817, 919]
[1, 181]
[1, 99, 231, 354, 488, 721, 867]
[1, 186, 416, 552, 830]
[1, 103, 413, 615, 649]
[1, 71, 103, 111, 289, 319, 390, 587, 657, 763]
[1, 65, 118, 235, 424, 462, 484, 710, 838]
[1, 74, 146, 288, 701]
[1, 252, 424]
[1, 30,

[324, 334, 612, 858, 913]
[324, 774]
[324, 596, 622]
[326, 828, 917]
[326, 620, 820]
[326, 561, 866]
[327, 849]
[328, 337]
[328, 643]
[328, 695, 707]
[328, 583, 698, 847]
[329, 629, 809, 828, 911, 917]
[329, 780]
[329, 813]
[330, 502]
[330, 830]
[330, 831]
[332, 334, 340, 766]
[332, 453, 786]
[332, 704]
[333, 387, 392]
[333, 594]
[333, 734]
[333, 709, 773]
[334, 338, 824]
[335, 794]
[335, 351]
[335, 754, 885]
[335, 658, 899]
[335, 799, 810, 883]
[335, 426]
[337, 400]
[337, 795]
[337, 346, 588, 589]
[338, 655]
[338, 592]
[338, 462, 493, 552]
[338, 582, 650, 674]
[338, 469, 903]
[338, 349]
[339, 389]
[339, 722, 837]
[339, 339]
[339, 914]
[339, 722]
[339, 652]
[340, 388, 395, 398, 505, 649]
[340, 818]
[341, 578, 685]
[341, 578]
[342, 869]
[342, 463, 586, 652, 696, 722]
[342, 525, 529, 543]
[342, 404, 616, 745, 755, 846]
[343, 381, 532, 781, 795]
[343, 411, 530, 709, 842]
[344, 770, 843]
[344, 400, 866]
[344, 765]
[344, 384, 530]
[344, 586]
[344, 402]
[345, 410, 661, 711, 738]
[345, 849]
[

In [35]:
news_data.loc[29]['news']

'Sens. Alan Cranston (D-Cal.) and Daniel Evans (R-Wash.) said they introduced export licensing reform legislation that could save U.S. companies hundreds of thousands of dollars annually.     "Our emphasis is two-fold: Decontrol and de-license items where such actions will not endanger our national security, and eliminate the Department of Defense\'s de facto veto authority over the licensing process," Cranston said.     "Our reforms should reduce licensing requirements by 65  to 70 pct," he told reporters. "I am convinced that a more rational...licensing process will boost exports."     U.S. export controls are intended to deny Eastern bloc countries access to technology that could further their military capabilities.     "By refocusing our control resources on higher levels of technology, technology that is truly critical, we will do a better job of preventing diversion of critical technology to our adversaries while promoting more exports," Cranston said.     "We cannot expect to co

In [36]:
news_data.loc[52]['news']

'Sens. Alan Cranston (D-Cal.) and Daniel Evans (R-Wash.) said they introduced export licensing reform legislation that could save U.S. companies hundreds of thousands of dollars annually.     "Our emphasis is two-fold: Decontrol and de-license items where such actions will not endanger our national security, and eliminate the Department of Defense\'s de facto veto authority over the licensing process," Cranston said.     "Our reforms should reduce licensing requirements by 65  to 70 pct," he told reporters. "I am convinced that a more rational...licensing process will boost exports."     U.S. export controls are intended to deny Eastern bloc countries access to technology that could further their military capabilities.     "By refocusing our control resources on higher levels of technology, technology that is truly critical, we will do a better job of preventing diversion of critical technology to our adversaries while promoting more exports," Cranston said.     "We cannot expect to co

## (4) Implement K-nearest neighbor (KNN) search using LSH and compare its performance with linear search.